In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from TrajectoryDataset import TrajectoryDataset
from DLModels import GraphTrajectoryLSTM
from Trainer import Trainer
from visualizer import visualize_predictions
import pickle
import os

def load_sequences(folder_path):
    all_sequences = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.pkl'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as f:
                sequences = pickle.load(f)
                all_sequences.extend(sequences)
    print(f"Loaded {len(all_sequences)} sequences")
    return all_sequences

def train():
    # Hyperparameters
    input_sizes = {
        'node_features': 4,
        'position': 2,
        'velocity': 2,
        'steering': 1,
        'object_in_path': 1,
        'traffic_light_detected': 1
    }
    hidden_size = 128
    num_layers = 2
    input_seq_len = 3  # past trajectory length
    output_seq_len = 3  # future prediction length
    batch_size = 128
    num_epochs = 10
    learning_rate = 0.001
    
    # Data loading
    data_folder = "Dataset/Sequence_Dataset"
    dataset = TrajectoryDataset(data_folder)
    
    # Split the dataset
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    
    print(f"Train set size: {len(train_dataset)}")
    print(f"Test set size: {len(test_dataset)}")
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)
    
    # Model initialization
    model = GraphTrajectoryLSTM(input_sizes, hidden_size, num_layers, input_seq_len, output_seq_len)
    
    # Training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trainer = Trainer(model, train_loader, test_loader, learning_rate, device)
    trained_model = trainer.train(num_epochs)
    
    # Save the trained model
    torch.save(trained_model.state_dict(), "graph_trajectory_model.pth")
    
    # Evaluate on test set
    test_loss = trainer.validate()
    print(f"Test Loss: {test_loss:.4f}")

    # Visualization
    #all_sequences = load_sequences(data_folder)
    visualize_predictions(trained_model, dataset, device)

def collate_fn(batch):
    past_batch = {k: torch.stack([item[0][k] for item in batch]) for k in batch[0][0].keys()}
    future_batch = {k: torch.stack([item[1][k] for item in batch]) for k in batch[0][1].keys()}
    graph_batch = {
        'node_features': torch.stack([item[2]['node_features'] for item in batch]),
        'adj_matrix': torch.stack([item[2]['adj_matrix'] for item in batch])
    }
    
    # Ensure all tensors have 3 dimensions
    for key in ['steering', 'object_in_path', 'traffic_light_detected']:
        if past_batch[key].dim() == 3:
            past_batch[key] = past_batch[key].squeeze(-1)
        if future_batch[key].dim() == 3:
            future_batch[key] = future_batch[key].squeeze(-1)
    
    # Print shapes for debugging
    #print("Shapes after collate_fn:")
    #for key, value in past_batch.items():
    #    print(f"past_{key}: {value.shape}")
    #for key, value in future_batch.items():
    #    print(f"future_{key}: {value.shape}")
    #for key, value in graph_batch.items():
    #    print(f"graph_{key}: {value.shape}")
    
    return past_batch, future_batch, graph_batch
train()

Loaded 5661 sequences
Train set size: 4528
Test set size: 1133


Training: 100%|██████████| 36/36 [00:11<00:00,  3.05it/s]


Epoch 1/10, Train Loss: 31.2772, Val Loss: 10.3339


Training: 100%|██████████| 36/36 [00:11<00:00,  3.08it/s]


Epoch 2/10, Train Loss: 9.6434, Val Loss: 9.6060


...

Training: 100%|██████████| 36/36 [00:12<00:00,  2.92it/s]


Epoch 6/10, Train Loss: 9.4460, Val Loss: 9.5865


Training: 100%|██████████| 36/36 [00:11<00:00,  3.05it/s]


Epoch 7/10, Train Loss: 9.4214, Val Loss: 9.6387


Training:   3%|▎         | 1/36 [00:00<00:11,  3.09it/s]